In [1]:
### imports
import json
import pandas as pd
from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
import re
import regex
tokenizer = TweetTokenizer()
from nltk.corpus import stopwords
stopW = stopwords.words('english')
# stopW = ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']
# print(sorted(stopW))
import pandas as pd
import math 

In [2]:
### a method to Read a json file with img captions and article titles and parse it into set of documents

def getDocs(sourcePath):
    medDocs = []
    sourceFile = open(sourcePath, 'r')

    count = 1
    for line in sourceFile:
        medDocs.append(json.loads(line))
        count += 1

    sourceFile.close()
    
    # for k in range(1,count):
    #     print(str(k) + " : " + medDocs[k]['docId'])
    return medDocs


sourcePath = 'assets/medImages.jl'
medDocs = getDocs(sourcePath)
medDocs[262]

{'articleId': 'PMC7282154',
 'image_path': 'PMC7282154-6_i2376-0605-6-2-e50-f06.jpg',
 'image_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/bin/i2376-0605-6-2-e50-f06.jpg',
 'articleUrl': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/',
 'articleTitle': 'RECURRENT INVASIVE DUCTAL BREAST CARCINOMA PRESENTING AS PRIMARY ADRENAL INSUFFICIENCY WITH ADRENAL CRISIS',
 'caption': 'Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.'}

In [3]:
# method that cleans the title and caption text in each document
# the result is tokenized titles and caption without unwanted brackets, numbers, spaces, tags, unicode charts;
# result includes no punctuation and stopwards and tokens with length less than 2 symbols
# all tokens are lower case


def preprocessContent(content):
    sentences = sent_tokenize(content)
    tokens = []
    for sent in sentences:
        # removes unicode
        cleanedTokens = re.sub(r'[^\x00-\x7F]+', ' ', sent)
        # removes menions
        cleanedTokens = re.sub(r'@\w+', '', cleanedTokens)
        # removes numbers
        cleanedTokens = re.sub('[0-9]+', '', cleanedTokens)
        # removes nested brackets 
        cleanedTokens = regex.sub(r'\([^()]*+(?:(?R)[^()]*)*+\)', '', cleanedTokens) 
        # removes nested currly brackets
        cleanedTokens = regex.sub('\{(?:[^}{]|\{[^}{]*\})*\}', '', cleanedTokens) 
        # removes html tags
        cleanedTokens = re.sub('\<+/*\w*/*\>+', '', cleanedTokens)
        # removes punctuation
        cleanedTokens = re.sub(r'[%s]' % re.escape(punctuation), ' ', cleanedTokens)
        # removes doubled spaces
        cleanedTokens = re.sub(r'\s{2,}', ' ', cleanedTokens)
        sentTokens = tokenizer.tokenize(cleanedTokens)
        # lower case each token and removes tokens with length less than 2
        sentTokens = [tok.lower() for tok in sentTokens if len(tok) >2]
        # removes stop words 
        sentTokens2 = [tok for tok in sentTokens if not tok.lower() in stopW ]
        
        ### implement stemming! play = played and/or stemming ?!?
        tokens += sentTokens2
    return tokens


def cleanDocs(medDocs):
    cleanedDocs = medDocs
    for i, doc in enumerate(cleanedDocs):
        doc['articleTitle'] = preprocessContent(doc['articleTitle'] )
        doc['caption'] = preprocessContent(doc['caption'])
 
    return cleanedDocs

cleanedDocs = cleanDocs(medDocs)

In [4]:
print(len(cleanedDocs))
# cleanedDocs[262]
# print(cleanedDocs[262]['articleTitle'])
print(cleanedDocs[262]['caption'])

263
['histologic', 'section', 'adrenal', 'metastatic', 'disease', 'tumor', 'cells', 'arranged', 'solid', 'nests', 'infiltrating', 'lymph', 'node', 'tissue']


In [5]:
### create a bag of words 
def createBagsOfWOrds(corpus):
    bagOfWordsTitle = []
    bagOfWordsCaption = []
    for doc in corpus:
        for term in doc['articleTitle']:
            if term not in bagOfWordsTitle:
                bagOfWordsTitle.append(term)
        for term in doc['caption']:
            if term not in bagOfWordsCaption:
                bagOfWordsCaption.append(term)
    
    return bagOfWordsTitle, bagOfWordsCaption

bagOfWordsTitle, bagOfWordsCaption = createBagsOfWOrds(cleanedDocs)

print(bagOfWordsTitle[:10], bagOfWordsCaption[:10])

['novel', 'use', 'stereotactic', 'radiotherapy', 'remnant', 'adrenal', 'tissue', 'nelson', 'syndrome', 'manifestations'] ['adrenal', 'radiation', 'planning', 'depicted', 'computed', 'tomography', 'scan', 'abdomen', 'planned', 'target']


In [6]:
# bow = set(bagOfWordsTitle).union(set(bagOfWordsCaption))
bow = bagOfWordsCaption
def computeFreq(corpus):
    termDicts = []
    for doc in corpus:
        termDicts.append(dict.fromkeys(bow,0))
        
    for i in range(len(termDicts)):
        for term in corpus[i]['caption']:
            if termDicts[i].get(term) != None:
                termDicts[i][term] +=1
    
    return termDicts

termDicts = computeFreq(cleanedDocs)
termDicts[0]

{'adrenal': 3,
 'radiation': 2,
 'planning': 1,
 'depicted': 1,
 'computed': 1,
 'tomography': 1,
 'scan': 1,
 'abdomen': 1,
 'planned': 1,
 'target': 2,
 'volume': 1,
 'sbrt': 2,
 'included': 2,
 'safety': 1,
 'margin': 1,
 'around': 1,
 'radiologic': 1,
 'abnormality': 1,
 'organs': 1,
 'risk': 1,
 'damage': 1,
 'treatment': 1,
 'bilateral': 1,
 'kidneys': 1,
 'spinal': 1,
 'cord': 1,
 'stomach': 1,
 'bowel': 1,
 'liver': 1,
 'image': 1,
 'shows': 2,
 'orientation': 1,
 'stereotactic': 2,
 'radiotherapy': 2,
 'beam': 1,
 'arcs': 1,
 'one': 2,
 'clockwise': 1,
 'counterclockwise': 1,
 'partial': 1,
 'arc': 1,
 'arrangement': 1,
 'utilized': 1,
 'treat': 1,
 'beds': 1,
 'treated': 1,
 'cgy': 1,
 'fractions': 1,
 'dose': 2,
 'color': 1,
 'wash': 1,
 'approximately': 1,
 'level': 1,
 'rapid': 1,
 'fall': 1,
 'excellent': 1,
 'sparing': 1,
 'oars': 1,
 'body': 1,
 'graph': 0,
 'changes': 0,
 'serum': 0,
 'cortisol': 0,
 'plasma': 0,
 'acth': 0,
 'levels': 0,
 'time': 0,
 'chronology': 0,


In [7]:
pd.DataFrame(termDicts)

,aaa,abbreviations,abdomen,abdominal,abel,ability,able,abnormal,abnormalities,abnormality,...,would,year,years,yellow,yielding,young,zannikos,zero,zidovudine,zonula
0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def computeTFperDoc(termDict, doc):
    tfDict = {}
    totalTermsCount = len(doc)
    for term, count in termDict.items():
        tfDict[term] = count/float(totalTermsCount)
    return tfDict

def computeTF(termDicts, corpus):
    tf = []
    N = len(corpus)
    for i, doc in enumerate(corpus):
        tf.append(computeTFperDoc(termDicts[i],doc))
    
    return tf
tfs = computeTF(termDicts, cleanedDocs)


In [9]:
pd.DataFrame(tfs)

,aaa,abbreviations,abdomen,abdominal,abel,ability,able,abnormal,abnormalities,abnormality,...,would,year,years,yellow,yielding,young,zannikos,zero,zidovudine,zonula
0,0.0,0.000000,0.166667,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.166667,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.166667,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [18]:
tfs[1]

{'adrenal': 0.0,
 'radiation': 0.0,
 'planning': 0.0,
 'depicted': 0.0,
 'computed': 0.0,
 'tomography': 0.0,
 'scan': 0.0,
 'abdomen': 0.0,
 'planned': 0.0,
 'target': 0.0,
 'volume': 0.0,
 'sbrt': 0.16666666666666666,
 'included': 0.16666666666666666,
 'safety': 0.0,
 'margin': 0.0,
 'around': 0.0,
 'radiologic': 0.0,
 'abnormality': 0.0,
 'organs': 0.0,
 'risk': 0.0,
 'damage': 0.0,
 'treatment': 0.16666666666666666,
 'bilateral': 0.0,
 'kidneys': 0.0,
 'spinal': 0.0,
 'cord': 0.0,
 'stomach': 0.0,
 'bowel': 0.0,
 'liver': 0.0,
 'image': 0.0,
 'shows': 0.16666666666666666,
 'orientation': 0.0,
 'stereotactic': 0.16666666666666666,
 'radiotherapy': 0.16666666666666666,
 'beam': 0.0,
 'arcs': 0.0,
 'one': 0.0,
 'clockwise': 0.0,
 'counterclockwise': 0.0,
 'partial': 0.0,
 'arc': 0.0,
 'arrangement': 0.0,
 'utilized': 0.0,
 'treat': 0.0,
 'beds': 0.0,
 'treated': 0.0,
 'cgy': 0.0,
 'fractions': 0.0,
 'dose': 0.0,
 'color': 0.0,
 'wash': 0.0,
 'approximately': 0.0,
 'level': 0.0,
 'rapi

In [10]:
def computeIDF(termDicts):
    idfDict = {}
    N = len(termDicts)
    idfDict = dict.fromkeys(termDicts[0].keys(), 0 )
    for term, val in idfDict.items():
        idfDict[term] = math.log10(N/(float(val)+1))
    return idfDict

idfs = computeIDF(termDicts)

In [14]:
###  compute TF - IDF
### TF-IDF = TF-IDF(title) * alpha + TF-IDF(body) * (1-alpha)
def computeTFIDF(tf, idfs):
    tfidf = {}
    for term, val in tf.items():
        tfidf[term] = val*idfs[term]
    return tfidf

tfidfs = []
for tf in tfs:
    tfidfs.append(computeTFIDF(tf,idfs))

In [15]:
tfidfs[0]
pd.DataFrame(tfidfs)

,aaa,abbreviations,abdomen,abdominal,abel,ability,able,abnormal,abnormalities,abnormality,...,would,year,years,yellow,yielding,young,zannikos,zero,zidovudine,zonula
0,0.0,0.000000,0.403326,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.403326,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.403326,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
